<a href="https://colab.research.google.com/github/Anurag07264/2022-web-developers/blob/main/Handgesture%20recognition%20project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
#!unzip archive.zip

In [1]:
import os
import cv2

# Define the base directory of the dataset
base_dir = './isl_data_grey_split/train'

# Define the labels for each category
labels = [str(i) for i in range(0, 10)] + [chr(i) for i in range(ord('a'), ord('z')+1)]

# Create a dictionary to map labels to numerical values
label_dict = {label: i for i, label in enumerate(labels)}

# Initialize an empty list to store the data
data = []

# Loop over each label in the current category
for label in labels:
    # Define the path to the current label
    path = os.path.join(base_dir, label)
    # Loop over each image in the current label directory
    for img in os.listdir(path):
        # Only process PNG images
        if img.endswith('.png'):
            # Define the path to the current image
            img_path = os.path.join(path, img)
            # Read the image
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            # Resize the image to 64x64 pixels
            image = cv2.resize(image, (64, 64))
            # Append the image and its numerical label to the data list
            data.append([image, label_dict[label]])

# At this point, the 'data' list contains all preprocessed images and their corresponding numerical labels
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming 'data' is your preprocessed data
images, labels = zip(*data)

# Convert the list to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize the images
images = images / 255.0

# Split the data into a training set and a test set
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Reshape the data to fit the model
train_images = train_images.reshape(-1, 64, 64, 1)
test_images = test_images.reshape(-1, 64, 64, 1)

# Create the model
model = Sequential()

# Add the layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(labels)), activation='softmax'))  # The number of nodes in the last layer should match the number of classes

# Compile the model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10)

# Save the model
model.save('my_model.h5')


Epoch 1/10
488/488 [==============================] - 86s 174ms/step - loss: 0.8457 - accuracy: 0.7620 - val_loss: 0.0423 - val_accuracy: 0.9892
Epoch 2/10
488/488 [==============================] - 82s 169ms/step - loss: 0.2209 - accuracy: 0.9267 - val_loss: 0.0213 - val_accuracy: 0.9941
Epoch 3/10
488/488 [==============================] - 86s 176ms/step - loss: 0.1669 - accuracy: 0.9418 - val_loss: 0.0156 - val_accuracy: 0.9956
Epoch 4/10
488/488 [==============================] - 89s 183ms/step - loss: 0.1283 - accuracy: 0.9537 - val_loss: 0.0144 - val_accuracy: 0.9951
Epoch 5/10
488/488 [==============================] - 85s 174ms/step - loss: 0.1117 - accuracy: 0.9600 - val_loss: 0.0103 - val_accuracy: 0.9969
Epoch 6/10
488/488 [==============================] - 84s 172ms/step - loss: 0.0922 - accuracy: 0.9652 - val_loss: 0.0092 - val_accuracy: 0.9972
Epoch 7/10
488/488 [==============================] - 83s 169ms/step - loss: 0.0865 - accuracy: 0.9686 - val_loss: 0.0105 - val_ac

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
import os
import cv2
from keras.models import load_model
# Define the base directory of the test dataset
base_dir = './isl_data_grey_split/test/'
# Define the labels for each category
labels = [str(i) for i in range(0, 10)] + [chr(i) for i in range(ord('a'), ord('z')+1)]

# Create a dictionary to map labels to numerical values
label_dict = {label: i for i, label in enumerate(labels)}

# Initialize an empty list to store the test data
test_data = []

# Loop over each label in the current category
for label in labels:
    # Define the path to the current label
    path = os.path.join(base_dir, label)
    # Loop over each image in the current label directory
    for img in os.listdir(path):
        # Only process PNG images
        if img.endswith('.png'):
            # Define the path to the current image
            img_path = os.path.join(path, img)
            # Read the image
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            # Resize the image to 64x64 pixels
            image = cv2.resize(image, (64, 64))
            # Append the image and its numerical label to the test data list
            test_data.append([image, label_dict[label]])

# At this point, the 'test_data' list contains all preprocessed test images and their corresponding numerical labels

# Load the model
model = load_model('my_model.h5')

# Separate the test images and labels
test_images, test_labels = zip(*test_data)

# Convert the list to numpy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize the test images
test_images = test_images / 255.0

# Reshape the test images to fit the model
test_images = test_images.reshape(-1, 64, 64, 1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_images, test_labels)

print(f'The accuracy of the model on the test set is {accuracy * 100:.2f}%')


153/153 [==============================] - 7s 44ms/step - loss: 0.0064 - accuracy: 0.9992
The accuracy of the model on the test set is 99.92%


In [ ]:
# import cv2
# import numpy as np
# from tensorflow.keras.models import load_model

# # Load the model
# model = load_model('my_model.h5')

# # Start the webcam
# cap = cv2.VideoCapture(0)

# while True:
#     # Capture frame-by-frame
#     ret, frame = cap.read()

#     if not ret:
#         break

#     # Convert the image to grayscale
#     image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Resize the image to 64x64 pixels
#     image = cv2.resize(image, (64, 64))

#     # Normalize the image
#     image = image / 255.0

#     # Reshape the image to fit the model
#     image = image.reshape(1, 64, 64, 1)

#     # Make a prediction
#     prediction = model.predict(image)

#     # Convert the prediction to a label
#     label = np.argmax(prediction, axis=1)

#     # Display the label on the frame
#     cv2.putText(frame, str(label), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

#     # Display the resulting frame
#     cv2.imshow('Sign Language Recognition', frame)

#     # Exit the loop if 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the capture
# cap.release()
# cv2.destroyAllWindows()


In [ ]:
!pip install flask-ngrok


In [ ]:
# from flask import Flask, render_template, request
# from flask_ngrok import run_with_ngrok
# import cv2
# import numpy as np
# from tensorflow.keras.models import load_model

# # Load the model
# model = load_model('my_model.h5')

# app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when the app is run

# @app.route('/', methods=['GET', 'POST'])
# def index():
#     if request.method == 'POST':
#         # Get the file from the POST request
#         file = request.files['file']

#         # Save the file to ./uploads
#         file.save('uploads/image.png')

#         # Load the image
#         image = cv2.imread('uploads/image.png', cv2.IMREAD_GRAYSCALE)

#         # Preprocess the image
#         image = cv2.resize(image, (64, 64))
#         image = image / 255.0
#         image = image.reshape(1, 64, 64, 1)

#         # Make a prediction
#         prediction = model.predict(image)

#         # Convert the prediction to a label
#         label = np.argmax(prediction, axis=1)

#         # Return the label
#         return str(label)

#     return render_template('index.html')

# if __name__ == '__main__':
#     app.run()


In [ ]:
# html = """
# <!DOCTYPE html>
# <html>
# <body>

# <h2>Upload File</h2>
# <form action="/" method="post" enctype="multipart/form-data">
#   Select image to upload:
#   <input type="file" name="file" id="file">
#   <input type="submit" value="Upload Image" name="submit">
# </form>

# </body>
# </html>
# """

# # Write the HTML content to 'templates/index.html'
# os.makedirs('templates', exist_ok=True)
# with open('templates/index.html', 'w') as f:
#     f.write(html)


In [15]:
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import io

# Load the model
model = load_model('my_model.h5')

# Create a FileUpload widget
uploader = widgets.FileUpload(
    accept='.png',  # Accepted file extension
    multiple=False  # True to accept multiple files upload else False
)

# Create a Button widget
button = widgets.Button(description="Make a prediction")

# Define the function to be called when the button is clicked
def on_button_clicked(b):
    # Get the uploaded file
    file = uploader.value[list(uploader.value.keys())[0]]

    # Convert the file to an image
    image = Image.open(io.BytesIO(file['content']))

    # Convert the image to a NumPy array
    image = np.array(image)

    # Preprocess the image
    image = cv2.resize(image, (64, 64))
    image = image / 255.0
    image = image.reshape(1, 64, 64, 1)

    # Make a prediction
    prediction = model.predict(image)

    # Convert the prediction to a label
    label = np.argmax(prediction, axis=1)

    # Print the label
    print(f"The predicted label is: {label}")

# Set the function to be called when the button is clicked
button.on_click(on_button_clicked)

# Display the widgets
display(uploader)
display(button)


FileUpload(value={}, accept='.png', description='Upload')

Button(description='Make a prediction', style=ButtonStyle())

1/1 [==============================] - 0s 100ms/step
The predicted label is: [0]
1/1 [==============================] - 0s 21ms/step
The predicted label is: [3]
